In [1]:
import numpy as np
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from sqlalchemy import create_engine
import psycopg2
import os
import plotly.figure_factory as ff
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

In [29]:
import plotly.express as px

In [2]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = 5432,
    db = 'congress'
))

In [3]:
myquery = '''
SELECT directordername, bioguideid
FROM members
ORDER BY lastname
'''

members = pd.read_sql_query(myquery, con=engine)
member_list = [{'label': x, 'value': y} for x, y in zip(members['directordername'], members['bioguideid'])]
member_list

[{'label': 'Alma S. Adams', 'value': 'A000370'},
 {'label': 'Robert B. Aderholt', 'value': 'A000055'},
 {'label': 'Pete Aguilar', 'value': 'A000371'},
 {'label': 'Mark Alford', 'value': 'A000379'},
 {'label': 'Rick W. Allen', 'value': 'A000372'},
 {'label': 'Colin Z. Allred', 'value': 'A000376'},
 {'label': 'Mark E. Amodei', 'value': 'A000369'},
 {'label': 'Kelly Armstrong', 'value': 'A000377'},
 {'label': 'Jodey C. Arrington', 'value': 'A000375'},
 {'label': 'Jake Auchincloss', 'value': 'A000148'},
 {'label': 'Brian Babin', 'value': 'B001291'},
 {'label': 'Don Bacon', 'value': 'B001298'},
 {'label': 'James R. Baird', 'value': 'B001307'},
 {'label': 'Troy Balderson', 'value': 'B001306'},
 {'label': 'Tammy Baldwin', 'value': 'B001230'},
 {'label': 'Becca Balint', 'value': 'B001318'},
 {'label': 'Jim Banks', 'value': 'B001299'},
 {'label': 'Andy Barr', 'value': 'B001282'},
 {'label': 'Nanette Diaz Barragan', 'value': 'B001300'},
 {'label': 'John Barrasso', 'value': 'B001261'},
 {'label':

In [4]:
members.columns

Index(['directordername', 'bioguideid'], dtype='object')

In [24]:
myquery = '''
SELECT bioguideid,
    directordername,
    party,
    ideology,
    state,
    district
FROM members
'''

ideo_df = pd.read_sql_query(myquery, con=engine)
ideo_df['chamber'] = ideo_df['district'].isnull()
ideo_df['chamber'] = ideo_df['chamber'].replace({True: 'Senate', False: 'House of Representatives'})

In [25]:
ideo_df

,bioguideid,directordername,party,ideology,state,district,chamber
0,B000944,Sherrod Brown,D,-0.427,Ohio,NaN,Senate
1,C000127,Maria Cantwell,D,-0.305,Washington,NaN,Senate
2,C000141,Benjamin L. Cardin,D,-0.320,Maryland,NaN,Senate
3,C000174,Thomas R. Carper,D,-0.178,Delaware,NaN,Senate
4,C001070,Robert P. Casey Jr.,D,-0.313,Pennsylvania,NaN,Senate
...,...,...,...,...,...,...,...
525,G000600,Marie Gluesenkamp Perez,D,-0.115,Washington,3.0,House of Representatives
526,V000135,Derrick Van Orden,R,0.381,Wisconsin,3.0,House of Representatives
527,H001096,Harriet M. Hageman,R,0.688,Wyoming,NaN,Senate
528,R000618,Pete Ricketts,R,0.369,Nebraska,NaN,Senate


In [37]:
ideo_df['namedistrict'] = [n + ' (' + s + ')' if c=='Senate' 
     else n + ' (' + s + '-' + str(int(d)) + ')' 
 for n,s,c,d in zip(ideo_df['directordername'], ideo_df['state'], ideo_df['chamber'], ideo_df['district'])]
ideo_df['party'] = ideo_df['party'].replace({
    'D': 'Democrat',
    'R': 'Republican',
    'I': 'Independent'
})



In [35]:
ideo_df

,bioguideid,directordername,party,ideology,state,district,chamber,namedistrict
0,B000944,Sherrod Brown,Democrat,-0.427,Ohio,NaN,Senate,Sherrod Brown (Ohio)
1,C000127,Maria Cantwell,Democrat,-0.305,Washington,NaN,Senate,Maria Cantwell (Washington)
2,C000141,Benjamin L. Cardin,Democrat,-0.320,Maryland,NaN,Senate,Benjamin L. Cardin (Maryland)
3,C000174,Thomas R. Carper,Democrat,-0.178,Delaware,NaN,Senate,Thomas R. Carper (Delaware)
4,C001070,Robert P. Casey Jr.,Democrat,-0.313,Pennsylvania,NaN,Senate,Robert P. Casey Jr. (Pennsylvania)
...,...,...,...,...,...,...,...,...
525,G000600,Marie Gluesenkamp Perez,Democrat,-0.115,Washington,3.0,House of Representatives,Marie Gluesenkamp Perez (Washington-3)
526,V000135,Derrick Van Orden,Republican,0.381,Wisconsin,3.0,House of Representatives,Derrick Van Orden (Wisconsin-3)
527,H001096,Harriet M. Hageman,Republican,0.688,Wyoming,NaN,Senate,Harriet M. Hageman (Wyoming)
528,R000618,Pete Ricketts,Republican,0.369,Nebraska,NaN,Senate,Pete Ricketts (Nebraska)


In [42]:
def ideograph(b):
    ideo2 = ideo_df.query(f"bioguideid == '{b}'")
    
    fig = px.scatter(ideo_df, x='ideology', y='chamber', color='party',
                     color_discrete_map={'Republican': 'red',
                                         'Democrat': 'blue',
                                         'Independent': 'green'},
                     height=300, width=600,
                     labels={'ideology':'Left/Right Ideology (DW-NOMINATE)', 
                            'chamber':'Chamber',
                            'party': 'Political Party'},
                     hover_name = 'namedistrict',
                     hover_data=['party'],
                     title = 'Ideological Placements in Congress',
                    opacity = .1)
    
     
    
    fig.update(layout=dict(title=dict(x=0.5)))
    fig.update_yaxes(range=(-0.5, 1.5), tickvals=[0, 1],
                    ticktext = ['Senate', 'House of Representatives'])
    fig.update_xaxes(range=(-1,1), tickvals=[-1, -.5, 0, .5, 1],
                    ticktext = ['Extreme left', 'Moderate left', 'Centrist', 'Moderate right', 'Extreme right'])
    
     
    
    fig.add_traces(
        px.scatter(ideo2, x='ideology', y='chamber', color='party',
                     color_discrete_map={'Republican': 'red',
                                         'Democrat': 'blue',
                                         'Independent': 'green'},
                     height=300, width=600,
                     labels={'ideology':'Left/Right Ideology (DW-NOMINATE)', 
                            'chamber':'Chamber',
                            'party': 'Political Party'},
                     hover_name = 'namedistrict',
                     hover_data=['party'],
                     title = 'Ideological Placements in Congress').update_traces(marker=dict(size=8, symbol="star", line=dict(width=2, color="DarkSlateGrey")),
                                                                                showlegend=False).data
    )
    
     
    
    
    return [fig]

In [28]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [7]:
mymarkdown = '''

Congressional elections are usually all about the big national issues. However, these Representatives and Senators work for local districts and states. Not all issues are the same everywhere. The purpose of this dashboard is to collect public data from these sources:

* [Official API for the U.S. Congres](https://api.congress.gov/)

* [Voteview](https://voteview.com)

* [Open Secrets](https://www.opensecrets.org/open-data/api)

* ProPublica's Data on Bills


'''

In [44]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        #Stuff on top

        html.H1("Know Thy Representatives in Congress"),
        html.H2("Data collected from APIs from Congress.gov, Open Secrets, Voteview.com, and Propublica"),
        html.H3("DS6600: Data Engineering 1, UVA Data Science"),

        #Side bar

        html.Div([
            dcc.Markdown('Select a Representative or Senator:'),
            dcc.Dropdown(id = 'member',
                         options = member_list,
                         value = 'A000370'),
            dcc.Markdown(mymarkdown)
        ], style = {'width': '24%', 'float': 'left'}),

        #Main bar
        html.Div([
            dcc.Tabs([
                    dcc.Tab(label = 'Biographical Info', children = [
                    # stuff for bio tab goes here
                    html.Div([html.Img(id = 'bioimage', style={'height':'100%', 'width':'100%'})], style = {'width': '24%', 'float':'left'}),
                    html.Div([dcc.Graph(id = 'biotable')], style = {'width': '74%', 'float':'right'})
                ]),
                dcc.Tab(label = 'Bill Sponsorship', children = [
                    #stuff for bill tab goes here
                ]),
                dcc.Tab(label = 'Voting and Ideology', children = [
                    #stuff for voting tab goes here
                    dcc.Graph(id = 'ideograph')
                ]),
                dcc.Tab(label = 'Donors and Financial Info', children = [
                    #stuff for donors tab goes here
                ])
            ])
        ], style = {'width': '74%', 'float': 'right'})
    ]
)

@app.callback([Output(component_id = 'biotable', component_property = 'figure')],
             [Input(component_id = 'member', component_property = 'value')])

def biotable(b):
    myquery = f'''
    SELECT directordername AS Name,
           party AS Party,
           state AS State,
           CAST(district AS int) AS District,
           birthyear AS Birthyear,
           addressinformation_officeaddress AS Address,
           CONCAT(addressinformation_city, ', ', addressinformation_district) AS City,
           addressinformation_zipcode AS zipcode,
           addressinformation_phonenumber
    FROM members
    WHERE bioguideid = '{b}'
    '''
    mydf = pd.read_sql_query(myquery, con=engine)
    mydf.columns = [x.capitalize() for x in mydf.columns]
    mydf = mydf.T.reset_index()
    mydf = mydf.rename({'index':'', 0: ''}, axis = 1)
    return [ff.create_table(mydf)]

@app.callback([Output(component_id = 'bioimage', component_property = 'src')],
             [Input(component_id = 'member', component_property = 'value')])
 

def bioimage(b):
    myquery = f'''
    SELECT depiction_imageurl
    FROM members
    WHERE bioguideid='{b}'
    '''
    mydf = pd.read_sql_query(myquery, con=engine)
    return [mydf['depiction_imageurl'][0]]

@app.callback([Output(component_id = 'ideograph', component_property = 'figure')],
             [Input(component_id = 'member', component_property = 'value')])

def ideograph(b):
    ideo2 = ideo_df.query(f"bioguideid == '{b}'")
    
    fig = px.scatter(ideo_df, x='ideology', y='chamber', color='party',
                     color_discrete_map={'Republican': 'red',
                                         'Democrat': 'blue',
                                         'Independent': 'green'},
                     height=300, width=600,
                     labels={'ideology':'Left/Right Ideology (DW-NOMINATE)', 
                            'chamber':'Chamber',
                            'party': 'Political Party'},
                     hover_name = 'namedistrict',
                     hover_data=['party'],
                     title = 'Ideological Placements in Congress',
                    opacity = .1)
    
     
    
    fig.update(layout=dict(title=dict(x=0.5)))
    fig.update_yaxes(range=(-0.5, 1.5), tickvals=[0, 1],
                    ticktext = ['Senate', 'House of Representatives'])
    fig.update_xaxes(range=(-1,1), tickvals=[-1, -.5, 0, .5, 1],
                    ticktext = ['Extreme left', 'Moderate left', 'Centrist', 'Moderate right', 'Extreme right'])
    
     
    
    fig.add_traces(
        px.scatter(ideo2, x='ideology', y='chamber', color='party',
                     color_discrete_map={'Republican': 'red',
                                         'Democrat': 'blue',
                                         'Independent': 'green'},
                     height=300, width=600,
                     labels={'ideology':'Left/Right Ideology (DW-NOMINATE)', 
                            'chamber':'Chamber',
                            'party': 'Political Party'},
                     hover_name = 'namedistrict',
                     hover_data=['party'],
                     title = 'Ideological Placements in Congress').update_traces(marker=dict(size=8, symbol="star", line=dict(width=2, color="DarkSlateGrey")),
                                                                                showlegend=False).data
    )
    
     
    
    
    return [fig]

if __name__ == "__main__":
    app.run_server(mode = 'external', host = "0.0.0.0", debug=True)


#Note:

Type 0.0.0.0:8050